In [ ]:
import io, os, sys
input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readline

def dfs(G, vis, i, u) :
  vis[u] = 1
  S = [u]
  while S :
    u = S.pop()
    for v in G[u] :
      if v == i or vis[v] : continue
      vis[v] = 1
      S.append(v)

def AP(G) :
  V = len(G)
  for i in range(V) :
    cnt = 0 # number of connected components
    vis = [0] * V
    for j in range(V) :
      if i == j or vis[j] : continue
      cnt += 1
      dfs(G, vis, i, j)
    if cnt > 1 : yield i

def sol() :
  V, E = map(int, input().split())
  G = [[] for _ in range(V)]

  for _ in range(E) :
    u, v = map(int, input().split())
    G[u-1].append(v-1)
    G[v-1].append(u-1)
  
  res = [*map(lambda x: str(x + 1), AP(G))]
  sys.stdout.write(f"{len(res)}\n{' '.join(res)}")
sol()

- naive한 구현 $O(V * (V+E))$

7 7
1 4
4 5
5 1
1 6
6 7
2 7
7 3
;;
6 7
1 2
2 3
3 4
4 5
5 1
5 6
6 2
;;
6 7
1 2
2 3
3 4
4 1
4 5
5 6
6 4
;;
20 15
6 2 
16 1 
6 1 
14 3 
20 19 
8 17 
3 7 
1 9 
14 20 
2 20 
19 7 
2 16 
11 8 
6 18 
15 11

In [ ]:
import io, os, sys
sys.setrecursionlimit(10 ** 5)
input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readline

def AP(G) :
  V = len(G)
  low = [0] * V # 현재 정점에서 도달 가능한 가장 낮은 노드의 pre 값
  pre = [0] * V # 전위 순회 번호
  art = [False] * V # 단절점 여부
  cnt = 0 #방문 순서 카운터

  def dfs(u, p) :
    nonlocal cnt
    cnt += 1
    low[u] = pre[u] = cnt
    child = 0 # 자식 수
    for v in G[u] :
      if not pre[v] :
        child += 1
        dfs(v, u)
        low[u] = min(low[u], low[v]) # low를 더 낮은 값으로 갱신한다.
        if low[v] >= pre[u] : #v가 역방향 간선일 경우 단절점이다.
          art[u] = True
      elif v != p : # 이미 방문한 경우 역방향 간선의 번호만 갱신한다.
        low[u] = min(low[u], pre[v])
    if p == -1 and child == 1 : # 루트 정점의 자식 개수가 2개 이상일 때에만 단절점이다.
      art[u] = False

  for u in range(V) :
    if not pre[u] :
      dfs(u, -1)
  return art

def sol() :
  V, E = map(int, input().split())
  G = [[] for _ in range(V)]

  for _ in range(E) :
    u, v = map(int, input().split())
    G[u-1].append(v-1)
    G[v-1].append(u-1)
  
  cnt = 0
  res = []
  for i, v in enumerate(AP(G)) :
    if not v: continue
    cnt += 1
    res.append(str(i+1))
  
  sys.stdout.write(f"{cnt}\n{' '.join(res)}")
sol()

- Iterative

In [ ]:
import io, os, sys
input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readline

def AP(G):
  V = len(G)
  D = [0] * V #discovery time
  low = [0] * V #lowest reachable vertex
  parent = [-1] * V 
  child = [0] * V
  AP = [False] * V

  cnt = 1
  s = [*range(V)]
  while s:
    v = s.pop()
    if v < 0:  # DFS subtree의 끝일 경우
      v = ~v  #원본 정점 번호
      u = parent[v]
      if u != -1:  # 루트 노드가 아니면
        child[u] += 1
        if low[v] >= D[u]: # u가 v의 부모이고, v의 자식들이 u보다 늦게 방문되었을 경우 단절점이다.
          AP[u] = True
        low[u] = min(low[u], low[v])
      elif u == -1 and child[v] == 1 : # 루트 노드이고, 자식이 1개일 경우 단절점이 아니다.
        AP[v] = False
      continue

    if D[v]: continue # 이미 방문한 노드일 경우

    D[v] = cnt
    low[v] = cnt
    cnt += 1

    s.append(~v) # 현재 끝난 것으로 표시
    for u in G[v]: #그리고 인접한 정점들을 스택에 넣는다.
      if not D[u]:
        parent[u] = v
        s.append(u)
      elif u != parent[v]:
        low[v] = min(low[v], D[u])

  res = []
  for i, v in enumerate(AP):
    if v: res.append(i)

  return res

def sol() :
  V, E = map(int, input().split())
  G = [[] for _ in range(V)]

  for _ in range(E) :
    u, v = map(int, input().split())
    G[u-1].append(v-1)
    G[v-1].append(u-1)
  
  res = AP(G)
  sys.stdout.write(f"{len(res)}\n{' '.join(map(lambda x: str(x + 1), res))}")
sol()